# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
import sqlite3
import pandas as pd

In [3]:
conn = sqlite3.Connection('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [7]:
cur.execute("""select * from orders""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,,181
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,,121
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141


In [11]:
cur.execute("""SELECT customerNumber, contactLastName, contactFirstName
                      FROM customers 
                      WHERE customerNumber in (SELECT customerNumber FROM orders 
                                               WHERE orderDate = '2003-01-31')""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [13]:
cur.execute("""SELECT * from orderdetails""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4


In [24]:
cur.execute("""SELECT * from products""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00


In [33]:
cur.execute("""SELECT productName, count(orderNumber
                      FROM products
                      WHERE productCode in (SELECT productCode from orderdetails)
                      GROUP BY productName;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,productName,productCode
0,18th Century Vintage Horse Carriage,S18_3136
1,18th century schooner,S24_2011
2,1900s Vintage Bi-Plane,S24_2841
3,1900s Vintage Tri-Plane,S24_4278
4,1903 Ford Model A,S18_3140
5,1904 Buick Runabout,S18_4522
6,1911 Ford Town Car,S18_2248
7,1912 Ford Model T Delivery Wagon,S24_3151
8,1913 Ford Model T Speedster,S18_2949
9,1917 Grand Touring Sedan,S18_1749


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [ ]:
cur.execute("""SELECT """)

## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [ ]:
#Your code here

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [ ]:
#Your code here

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!